<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [2]:
! pip install polars


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import polars as pl
import random
import string


# Generacion de Datos

In [4]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""nlloj""",4,86.518592,"2016-01-11 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1699
2,"""aybip""",6,58.404277,"2012-07-16 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4263
3,"""hjkps""",7,48.286182,"2012-07-15 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1090
4,"""ojbxo""",15,16.235841,"2015-02-13 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3877
5,"""opvvo""",11,64.356533,"2011-09-24 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1208


# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [13]:
df = df.lazy()
# Realizar todas las operaciones de limpieza 
resultado = (
    df
    .filter(pl.col("cantidad") >= 5)
    .with_columns((pl.col("cantidad") * pl.col("precio_unitario")).alias("total_venta"))
   
    .drop("precio_unitario")
    .group_by("producto")
    .agg([
        pl.sum("total_venta").alias("total_vendido"),
        pl.sum("cantidad").alias("cantidad_total_vendida"),
        
    ])
    .sort("total_vendido", descending = True)
)

  

#mostrar los resultados
result_df = resultado.collect()
print(result_df)

shape: (805, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ cnyce    ┆ 1980.154894   ┆ 20                     │
│ qguna    ┆ 1964.542583   ┆ 20                     │
│ wmsja    ┆ 1962.30905    ┆ 20                     │
│ msviy    ┆ 1955.266084   ┆ 20                     │
│ jkztr    ┆ 1932.440729   ┆ 20                     │
│ …        ┆ …             ┆ …                      │
│ eigta    ┆ 83.577294     ┆ 8                      │
│ zbukl    ┆ 78.887682     ┆ 6                      │
│ ffusd    ┆ 75.276996     ┆ 7                      │
│ rjrgs    ┆ 70.777219     ┆ 7                      │
│ prsaw    ┆ 65.776784     ┆ 6                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [14]:
# Tu codigo aqui
result_df = resultado.collect()

# Guardar el DataFrame en un archivo Parquet
result_df.write_parquet("result.parquet")

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [15]:
# Tu codigo aqui
df = pl.read_parquet("result.parquet")

# Mostrar las primeras filas del DataFrame
print(df.head())

shape: (5, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ cnyce    ┆ 1980.154894   ┆ 20                     │
│ qguna    ┆ 1964.542583   ┆ 20                     │
│ wmsja    ┆ 1962.30905    ┆ 20                     │
│ msviy    ┆ 1955.266084   ┆ 20                     │
│ jkztr    ┆ 1932.440729   ┆ 20                     │
└──────────┴───────────────┴────────────────────────┘
